In [1]:
import importlib, fnmatch


import src.valid_index
importlib.reload(src.valid_index)
from src.valid_index import get_banking_act_index

import src.file_tools
importlib.reload(src.file_tools)
from src.file_tools import read_processed_regs_into_dataframe, get_regulation_detail

import src.embeddings
importlib.reload(src.embeddings)
from src.embeddings import get_ada_embedding, num_tokens_from_string

import src.tree_tools
importlib.reload(src.tree_tools)
from src.tree_tools import build_tree_for_regulation, split_tree


index_reg23 = get_banking_act_index()

non_text_labels = ['Table', 'Formula', 'Example', 'Definition']
dir_path = './txt/'
file_list = []
for root, dir, files in os.walk(dir_path):
    for file in files:
        str = 'reg23_sections*.txt'
        if fnmatch.fnmatch(file, str):
            file_path = os.path.join(root, file)
            file_list.append(file_path)

df_reg23, non_text = read_processed_regs_into_dataframe(file_list=file_list, valid_index_checker=index_reg23, non_text_labels=non_text_labels)
tree_reg23 = build_tree_for_regulation("BA", df_reg23, valid_index_checker=index_reg23)


section_summary_with_embeddings = "./tmp/summary_reg23_with_embedding.parquet"
section_questions_with_embeddings = "./tmp/summary_reg23_questions_with_embedding.parquet"
headings_index_file = "./tmp/headings_reg23.csv"




In [2]:
import pandas as pd
import os

sectioned_df = pd.DataFrame([],columns = ["section", "text", "token_count"])
save_sectioned_df_to_file = "./tmp/banking_act.csv"
exists = os.path.exists(save_sectioned_df_to_file)

if not exists:
    print("Loading the initial split of the tree. You will need to make changes to this as you see the data")
    # Starting at an particular parent node (can be the tree root or any child), this method splits up the 
    # branch into sections where the text does not exceed a certain word_count cap.
    sectioned_df = split_tree(tree_reg23.root, df_reg23, 700, index_reg23)
else:
    print("Loading the currency split of the tree so you can continue generating summaries and questions")
    sectioned_df = pd.read_csv(save_sectioned_df_to_file, encoding="utf-8", sep="|")

print(f'Total number of sections: {len(sectioned_df)}')


Loading the initial split of the tree. You will need to make changes to this as you see the data
Total number of sections: 461


Create or load the DataFrames that will hold the text index. Later we will add the embeddings to the same DataFrames. When we do so, loading the embeddings is slow from certain file formats like csv so we just start with a fast loading file format - parquet 

In [3]:
df_summary = None
if os.path.exists(section_summary_with_embeddings):
    df_summary = pd.read_parquet(section_summary_with_embeddings, engine='pyarrow')
    print(f"Summary data contains {len(df_summary)} lines of text")
    missing = len(df_summary[df_summary["text"] == ""])
    if missing > 0:
        print(f" -- of which there are {missing} lines that do not contain index text (e.g. sections with only definitions or indexes)")
else:
    print("Creating a new summary DataFrame")
    df_summary = pd.DataFrame([], columns = ["text", "section"])


df_questions = None
if os.path.exists(section_questions_with_embeddings):
    df_questions = pd.read_parquet(section_questions_with_embeddings, engine='pyarrow')
    print(f"Questions data contains {len(df_questions)} lines of text")    
    missing = len(df_questions[df_questions["text"] == ""])
    if missing > 0:
        print(f" -- of which there are {missing} lines that do not contain index text (e.g. sections with only definitions or indexes)")
else:
    print("Creating a new questions DataFrame")
    df_questions = pd.DataFrame([], columns = ["text", "section"])

index = None
if len(df_summary) != len(df_questions):
    print("The summary and the questions DataFrames do not have the same length")
else:
    index = len(df_summary)
    p = (index / len(sectioned_df)) * 100
    print(f'There are a total number of {len(sectioned_df)} sections to index')
    print(f"You have created {p:.2f} percent of your text index")


Summary data contains 9 lines of text
Questions data contains 9 lines of text
There are a total number of 461 sections to index
You have created 1.95 percent of your text index


In [35]:
#index = 120
print(get_regulation_detail("23(5)(b)(i)", df_reg23, index_reg23))

(5) Calculation of credit risk exposure: standardised approach
Subject to the relevant provisions of regulation 38(2) and subregulation (20), a bank that adopted the standardised approach for the measurement of the bank's exposure to credit risk
    (b) shall in a consistent manner, in accordance with the relevant requirements specified below, and in terms of the bank 's internal risk management process, apply the ratings or assessments issued by an eligible external credit assessment institution of the bank's choice, or export credit agency, to calculate the bank's risk exposure in terms of the relevant provisions contained in these Regulations, that is, the bank shall not "cherry pick" ratings or assessments issued by different external credit assessment institutions, arbitrarily change the use of eligible external credit assessment institutions of apply ratings of assessments for purposes of these Regulations differently from the bank's Internal risk management process.
        (i) 

In [16]:
from openai import OpenAI
client = OpenAI()
system_content_summerise = "You are summarising parts of Regulation 23 of the Banks Act (Reg23) for a bank that needs to complete the Credit Risk Monthly Return (Form BA 200). When summerising, do not add filler words like 'the act says ...' or 'Reg23 says ...', just summarise the section. Your summary should use plain language and avoid legalese. Since it is for a bank, when the act uses the phrase 'bank', please replace it with the relevant first person pronoun like 'I' or 'me'. A good summary will minimise the use of lists. Rather it should make use of paragraphs.\n\
Note: Reg23 offers two approaches to modelling credit risk namely the standardised approach and the internal ratings-based (IRB) approach. The standardised approach is subdivided into the 'Simplified Standardised' and the 'Standardised' approach. The IRB approach is subdivided into the 'Foundation IRB' (FIRB) and the 'Advanced IRB' (AIRB). The act will often refer to Method 1 or Method 2 but please use the full name or acronym of the appropriate calculation methodology in your summary."

user_context = get_regulation_detail("23(7)(e)", df_reg23, index_reg23)
response = client.chat.completions.create(
                    model=model,
                    temperature = 1.0,
                    max_tokens = 500,
                    messages=[
                        {"role": "system", "content": system_content_summerise},
                        {"role": "user", "content": user_context},
                    ]
                )
response

ChatCompletion(id='chatcmpl-8QAEtDiyP3H76dRmMRN5bBm2kbI6w', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="In the simplified standardised approach, if I obtain eligible collateral, guarantees, or a netting agreement that effectively transfers risk, I can reduce my credit risk exposure. Any transaction with credit protection cannot be assigned a higher risk weight than a similar transaction without protection.\n\nFor securitisation exposure, I can only recognize protection if it meets certain conditions. Eligible collateral must qualify as per specific provisions, and guarantees and credit-derivative instruments can only come from approved providers that meet minimum requirements. However, special-purpose institutions connected to the securitisation cannot be considered eligible protection providers. For the portions of exposure that are protected and unprotected, I need to maintain capital requirements following the set requirements.\n\nWhen deali

In [18]:
response.choices[0].finish_reason

'stop'

## Create the text index 

This is a manual process. We call OpenAI and print out the answers in a format that is used to update the index text but this does need to be edited before it is added to the index so this is not automated.


In [9]:
import src.summarise_and_question
importlib.reload(src.summarise_and_question)
from src.summarise_and_question import get_summary_and_questions_for


#model = "gpt-3.5-turbo"
#model="gpt-4"
model="gpt-4-1106-preview"

reg_text = sectioned_df.loc[index]['text']
print("##############")
print(reg_text)
print("##############")

model_summary, model_questions = get_summary_and_questions_for(reg_text, model = model)

#format output
section = sectioned_df.loc[index]['section']
print(f'df_summary.loc[index, "section"] = "{section}"')
print(f'df_summary.loc[index, "text"] = "{model_summary}"')
print()
print(f'df_questions.loc[index, "section"] = "{section}"')
print(f'df_questions.loc[index, "text"] = "{model_questions}"')



##############
(5) Calculation of credit risk exposure: standardised approach
Subject to the relevant provisions of regulation 38(2) and subregulation (20), a bank that adopted the standardised approach for the measurement of the bank's exposure to credit risk
    (d) shall comply with the relevant requirements specified in subregulations (6) to (9) below
##############
df_summary.loc[index, "section"] = "23(5)(d)"
df_summary.loc[index, "text"] = "Using the Standardised approach to measure my credit risk exposure, I must comply with certain requirements specified in sections (6) to (9) of Regulation 23, as long as they align with the provisions in regulation 38(2) and section (20)."

df_questions.loc[index, "section"] = "23(5)(d)"
df_questions.loc[index, "text"] = "What approach should I follow to measure credit risk exposure according to Regulation 23? | In complying with Regulation 23, which sections outline the requirements for measuring credit risk using the Standardised approach?"

In [10]:
# Once the last four lines are manually checked, the edited result is copied here and the summary and question index is updated

df_summary.loc[index, "section"] = "23(5)(d)"
df_summary.loc[index, "text"] = "Using the Standardised approach to measure my credit risk exposure, I must comply with certain requirements specified in sections (6) to (9) of Regulation 23."

df_questions.loc[index, "section"] = "23(5)(d)"
df_questions.loc[index, "text"] = "Which sections outline the requirements for measuring credit risk using the Standardised approach?"




index = index + 1
if index == len(sectioned_df):
    print("All done!")
else:
    next_section = sectioned_df.iloc[index]["section"]
    assert len(sectioned_df[sectioned_df["section"] == next_section]) == 1, "Huston, we have a problem"
    print(f'Next section is {next_section} which is on line {index}')
    p = ((index) / len(sectioned_df)) * 100
    print(f"You have completed {p:.2f} percent of your work")
    reg_text = sectioned_df.loc[index]['text']
    print("Next section")
    print("##############")
    print(reg_text)
    print("##############")



Next section is 23(6)(a) which is on line 12
You have completed 2.60 percent of your work
Next section
##############
(6) Method 1: Calculation of credit risk exposure in terms of the simplified standardised approach
Unless specifically otherwise provided in these Regulations, a bank that adopted the simplified standardised approach for the measurement of the bank's exposure to credit risk arising from positions held in its banking book shall risk weight its relevant exposure, net of any credit impairment, in accordance with the relevant requirements specified below:
    (a) In the case of exposure to sovereigns, central banks, public-sector entities, banks, securities firms and corporate institutions, in accordance with the provisions of table 1 below.
##############


In [11]:
# Sometimes there are errors in the previous code block. We need to be careful when saving over any work we have already done so the 
# save step is a manual one which needs to be run regularly but without overwriting good data with bad data
df_summary.to_parquet(section_summary_with_embeddings, engine='pyarrow')
df_questions.to_parquet(section_questions_with_embeddings, engine='pyarrow')


In [77]:
#df_summary.drop(49, inplace=True)
df_summary[134:145]

#df_questions[40:55]

,text,section
135,The Minister has the authority to establish fe...,111.
136,The Minister has the authority to make regulat...,112.
137,"Before creating or changing regulations, the M...",113.
138,You have one year from the commencement of thi...,114.
139,The name of the Act is the Protection of Perso...,115.


# NOTE: Some of the values in df_summary are pipe separated

## Changing how the sections are chunked

From time to time we will see instances whe the initial chunk size needs to be adjusted and nodes need to be expanded or collapsed. We do this in two stages. First we can experiment with a new "token_limit_per_chunk" for the node in question and once we find the chunking solution we are after, we remove the old chunks and replace them with the new chunks.


In [91]:
node_str = "C.1(D)(v)"
# get the list of indicies that start with this string
index_list = sectioned_df.index[sectioned_df['section'].str.startswith(node_str)].tolist()
is_consecutive = all(x+1 == y for x, y in zip(index_list[:-1], index_list[1:]))
assert is_consecutive, "The list of indicies that start with the node string is not consecutive so the rest of the logic here will not hold"
start_index_to_replace = index_list[0]
end_index_to_replace = index_list[-1] + 1
print(f"This will remove {len(index_list)} row(s) from sectioned_df. From {start_index_to_replace} to {end_index_to_replace}")

# Get the new set of indicies assuming a different chunking length
node = tree_reg23.get_node(node_str)
token_limit_per_chunk = 1300

tmp_df = split_tree(node, df_reg23, token_limit_per_chunk, index_reg23)

print(f"... and will replace them with {len(tmp_df)} row(s)")
print(tmp_df)


This will remove 9 row(s) from sectioned_df. From 82 to 91
... and will replace them with 1 row(s)
     section                                               text  token_count
0  C.1(D)(v)  C.1 FinSurv Reporting System\n    (D) Offshori...         1208


Replace the node and all its children with the new DataFrame with a different word_count limit

In [92]:
def replace_rows(original_df, updated_section_df, start_row, end_row):
    before = original_df.iloc[:start_row]
    after = original_df.iloc[end_row:]
    new_df = pd.concat([before, updated_section_df, after]).reset_index(drop=True)
    return new_df

print(f"The original data consisted of {sectioned_df} chunks")
sectioned_df = replace_rows(sectioned_df, tmp_df, start_row=start_index_to_replace, end_row=end_index_to_replace)
print(f"Post the update, the data consists of {sectioned_df} chunks")

Jesus saves! 
But only if he is happy with the results. Check first!

In [135]:
sectioned_df

,section,text,token_count
0,2.,2. Purpose of Act\nThe purpose of this Act is ...,200
1,3.,3. Application and interpretation of Act\n ...,379
2,4.,4. Lawful processing of personal information\n...,655
3,5.,5. Rights of data subjects\nA data subject has...,472
4,6.,6. Exclusions\n (1) This Act does not apply...,279
...,...,...,...
135,111.,"111. Fees\n (1) The Minister may, subject t...",111
136,112.,"112. Regulations\n (1) The Minister may, su...",423
137,113.,113. Procedure for making regulations\n (1)...,380
138,114.,114. Transitional arrangements\n (1) All pr...,207


In [78]:
sectioned_df.to_csv(save_sectioned_df_to_file, encoding="utf-8", sep="|", index = False)

## Definitions

In [99]:
import re
import pandas as pd

import src.embeddings
importlib.reload(src.embeddings)
from src.embeddings import get_ada_embedding, num_tokens_from_string

def count_leading_spaces(s):
    match = re.match(r'^\s*', s)  # Matches leading whitespace
    return len(match.group(0))

add_embeddings = False
definitions_to_process = '#Definition 1'
if definitions_to_process == '#Definition 1':
    popia_definitions_and_embeddings_file = "./tmp/popia_definitions_with_embeddings.csv"
    exclude_first_line = True
    start_line = 2
else:
    raise NotImplemented("Only implemented for popia Definition 1")

popia_manual_definitions = []
#raw_list = non_text['Definition']['#Definition 1']
raw_list = non_text['Definition'][definitions_to_process]
number_of_spaces = count_leading_spaces(raw_list[start_line])
if number_of_spaces % 4 != 0:
    raise ValueError(f"This line does not have an indent which is a multiple of 4: {raw_list[start_line]}")

current_line = raw_list[start_line]

current_line_number_of_spaces = count_leading_spaces(current_line)
if current_line_number_of_spaces != number_of_spaces:
    print(f"current_line_number_of_spaces: {current_line_number_of_spaces}")
    print(f'number_of_spaces: {number_of_spaces}')
    raise ValueError(f"This line does not have the correct indentation: {current_line}")

processing_table = False
for line_number in range(start_line,len(raw_list)-1):
    next_line = raw_list[line_number + 1]
    next_line_number_of_spaces = count_leading_spaces(next_line)
    if next_line_number_of_spaces % 4 != 0:
        raise ValueError(f"This line does not have an indent which is a multiple of 4: {next_line_number_of_spaces}")

    if current_line_number_of_spaces == next_line_number_of_spaces:
        current_line = current_line.lstrip()
        if "|" in current_line: # processing something with table formatting
            processing_table = True
            split_line = [x.strip() for x in current_line.split("|")]
            popia_manual_definitions.append(split_line)
        else:
            popia_manual_definitions.append(current_line)
        current_line = next_line
    else:
        current_line = current_line + "\n" + next_line

# add the last entry
current_line = current_line.lstrip()
if processing_table:
    split_line = [x.strip() for x in current_line.split("|")]
    popia_manual_definitions.append(split_line)
else:
    popia_manual_definitions.append(current_line.lstrip())

if processing_table:
    headings = popia_manual_definitions[0]
    popia_manual_definitions.pop(0)
    df_reg23_definitions = pd.DataFrame(popia_manual_definitions, columns=headings) 
else:
    df_reg23_definitions = pd.DataFrame(popia_manual_definitions, columns=["Definition"])


if add_embeddings:
    df_reg23_definitions["Embedding"] =df_reg23_definitions["Definition"].apply(get_ada_embedding)

df_reg23_definitions["source"] = 'all'

#df_reg23_definitions.to_csv(popia_definitions_and_embeddings_file, encoding = "utf-8", sep = "|", index = False)
df_reg23_definitions.to_parquet("./inputs/popia_definitions.parquet", engine='pyarrow')


OSError: Cannot save file into a non-existent directory: 'inputs'

In [136]:
df_reg23_definitions = pd.read_parquet("./inputs/popia_definitions.parquet", engine='pyarrow')

In [138]:
df_reg23_definitions['source'] = 'all'

In [139]:
df_reg23_definitions.to_parquet("./inputs/popia_definitions.parquet", engine='pyarrow')

## Section headings are also a good index

In [105]:
# step 1 is to remove all text that is not tagged as a heading leaving only the index and headings
toc_file = "./tmp/section_numbers_and_headings.txt" # Note this is a temporary file and will be deleted in a few cells time
df = df_reg23
index = index_reg23


written_references = set() # only write each reference once
with open(toc_file, 'w', encoding = 'utf-8') as f:
    for _, row in df.iterrows():
        if row['full_reference'] not in written_references:
            written_references.add(row['full_reference'])
            s = ' ' * row['Indent'] * 4 + row['Reference']
            # skip the text for into and legal
            if row['Heading'] and (row['Indent'] == 0 and row['Text'].strip() in index.exclusion_list):
                s = s
            elif row['Heading']:               
                s += " " + row['Text']
            f.write(s + '\n')

In [106]:
# step 2) Remove all lines that do not have text. Since the only text is for the headers, this removes everything that is not a header.
#         Note however that the remaining "headers" will not contain the (#headers) markdown so will be treated as text
import src.file_tools
importlib.reload(src.file_tools)
from src.file_tools import process_regulations

excon_headers = './tmp/non_empty_headings_excon.txt'  # Note this is a temporary file and will be deleted in a few cells time
files_as_list = []
files_as_list.append(toc_file)
df_toc, non_text_toc = process_regulations(files_as_list, valid_index_checker=index_reg23, non_text_labels=non_text_labels)

for index, row in df_toc.iterrows():
    if row['Text'] != "":
        df_toc.at[index, 'Heading'] = True

tree_toc = build_tree_for_regulation("popia_toc", df_toc, valid_index_checker=index_reg23)
l = tree_toc._list_node_children(tree_toc.root)
with open(excon_headers, 'w', encoding = 'utf-8') as f:
    f.write(l)

In [107]:
# step 3) The file that contains the headers (now as text because they are missing the (#heading) markdown) and load it up
#         as if it were the regs themselves
files_as_list = []
files_as_list.append(excon_headers)
df_non_empty_toc, non_text_toc = process_regulations(files_as_list, valid_index_checker=index_reg23, non_text_labels=non_text_labels)
for index, row in df_non_empty_toc.iterrows():
    if row['Text'] != "":
        df_non_empty_toc.at[index, 'Heading'] = True

non_empty_tree_toc = build_tree_for_regulation("Excon", df_non_empty_toc, valid_index_checker=index_reg23)


In [108]:
# Construct the full reference of each heading and use these as a key for a dictionary where the heading is the value
import os

def get_leaf_headings(root):
    leaf_headings = {}

    def recurse(node, heading):
        # Add ". " only if it's not the root node and the node's heading_text is not empty
        new_heading = heading + (". " + node.heading_text if heading and node.heading_text else node.heading_text)
        if not node.children:  # This is a leaf node.
            leaf_headings[node.full_node_name] = new_heading
        else:  # This is not a leaf node. We continue the recursion.
            for child in node.children:
                recurse(child, new_heading)

    recurse(root, '')  # We start the recursion from the root, with an empty heading.
    return leaf_headings

leaf_headings = get_leaf_headings(non_empty_tree_toc.root)
df_section_headings = pd.DataFrame(list(leaf_headings.items()), columns=['section', 'text'])
print(f'Popia Act contains {len(leaf_headings)} section headings')




df_section_headings.to_csv(headings_index_file, encoding = "utf-8", sep = "|", index = False)
if os.path.exists(toc_file):
    os.remove(toc_file)
if os.path.exists(excon_headers):
    os.remove(excon_headers)



Popia Act contains 114 section headings


In [109]:
df_index.to_parquet("./inputs/popia_index.parquet", engine='pyarrow')
# NOTE: At some point the heading sections were given an additional "0" at the end of the section which causes issues.
#        Confirm all the headings are "2." and not "2.0"

,section,text
0,2.,Purpose of Act
1,3.,Application and interpretation of Act
2,4.,Lawful processing of personal information
3,5.,Rights of data subjects
4,6.,Exclusions
...,...,...
109,111.,Fees
110,112.,Regulations
111,113.,Procedure for making regulations
112,114.,Transitional arrangements


In [116]:
# Index

df_index = pd.read_parquet(section_questions_with_embeddings, engine='pyarrow')
df_index = df_index[df_index["text"] != ""] # remove rows that have 'text' == ""
# the 'text' column for the questions may contain multiple questions separated by a "|". The next line expands these rows
# so the value in 'text' only contains one question
df_index = df_index.drop("text", axis=1).join(df_index["text"].str.split("|", expand=True).stack().reset_index(level=1, drop=True).rename("text"))
df_index.reset_index(drop=True, inplace=True)
df_index["source"] = "question"

df_tmp = pd.read_parquet(section_summary_with_embeddings, engine='pyarrow')
df_tmp["source"] = "summary"

df_tmp_2 = pd.read_csv(headings_index_file, encoding = "utf-8", sep = "|")
df_tmp_2["source"] = "heading"

df_index = pd.concat([df_index, df_tmp, df_tmp_2], ignore_index = True)
df_index = df_index[df_index["text"]!= ""]
df_index = df_index[df_index["text"].notna()] # Remove any NaN's
df_index.reset_index(drop=True, inplace=True)

#df_index['Embedding'] = df_index['text'].apply(get_ada_embedding)


In [126]:
df_index['section'] = df_index['section'].astype(str)
df_index['text'] = df_index['text'].astype(str)
df_index['source'] = df_index['source'].astype(str)

In [120]:
df_index['Embedding'] = df_index['text'].apply(get_ada_embedding)

In [128]:
df_index.dtypes

section      object
text         object
source       object
Embedding    object
dtype: object

In [129]:
df_index.to_parquet("./inputs/popia_index.parquet", engine='pyarrow')


In [130]:
df_tmp = pd.read_parquet("./inputs/popia_index.parquet", engine='pyarrow')

In [131]:
df_tmp

,section,text,source,Embedding
0,2.,What is the purpose of the Protection of Perso...,question,"[-0.00897529348731041, -0.004865928087383509, ..."
1,2.,What rights does the Act provide individuals ...,question,"[-0.0016322160372510552, 0.0023074529599398375..."
2,2.,What kind of regulatory body is established b...,question,"[0.004339318256825209, -0.012121826410293579, ..."
3,3.,What data or information does POPIA apply to?,question,"[0.0023713144473731518, 0.0032313510309904814,..."
4,3.,How does the POPIA interact with other laws r...,question,"[0.003651085076853633, 0.010420947335660458, 0..."
...,...,...,...,...
481,111.0,Fees,heading,"[-0.0012755942298099399, 0.006350830662995577,..."
482,112.0,Regulations,heading,"[0.0018349532037973404, -0.012941248714923859,..."
483,113.0,Procedure for making regulations,heading,"[0.009133221581578255, 0.004106280393898487, -..."
484,114.0,Transitional arrangements,heading,"[0.00048234881251119077, -0.011674447916448116..."


In [26]:
# if there is an error somewhere in the generation of the embedding and you need to find it, this is a hacky way to do that
increment = 10
for i in range(0, len(df_index), increment):
    chunk = df_index.iloc[i:i+increment].copy()
    chunk["Embedding"] = chunk["text"].apply(get_ada_embedding)
    df_index.loc[chunk.index, "Embedding"] = chunk["Embedding"]
    print(f"Completed {i+increment} lines")


#df_index


Completed 250 lines
Completed 260 lines
Completed 270 lines
Completed 280 lines
Completed 290 lines
Completed 300 lines
Completed 310 lines
Completed 320 lines
Completed 330 lines
